# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [117]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [179]:
import time
def variable_depth_local_search(graph):
    def get_weight(left):
        ans = 0
        for edge in edges:
            a, b = edge
            if a in left and not b in left:
                ans += 1
        return ans

    def move(ch1, a, b):
        if (a in ch1[0]):
            ch1[0].discard(a)
        else:
            ch1[1].add(a)
        if (b in ch1[1]):
            ch1[1].discard(b)
        else:
            ch1[0].add(b)
                
    time_start = time.monotonic()
    cnt = 0
    x = set()
    
    starting_point = set(range(1, len(graph[0]) // 2 + 1))
    vertices, edges = graph
    n = len(vertices)
    G = [[] for i in range(1 + n)]
    for edge in edges:
        a, b = edge
        G[a] += [b]
    left = list(starting_point)
    bestleft = left[:]
    right = list(range(len(graph[0]) // 2 + 1, len(graph[0]) + 1))
    rec = 5 #длина прыжка
    opt = get_weight(left)
    r = 0
    parent = (-1, -1)
    cur = opt
    before = [] #будем во время прыжка хранить посещенные состояния
    ch = [set(), set()] #добавленные в левую и правую доли, описывают разбиение с момента начала длинного прыжка
    cost_left = [0 for i in range(n+1)]
    cost_right = [0 for i in range(n+1)]
    for edge in edges:
        a, b = edge
        if b in left:
            cost_left[a] += 1
        else:
            cost_right[a] += 1
    while (True):
        cnt += 1
        time_elapsed = time.monotonic()-time_start
        if time_elapsed > 45:
            break
        if rec == r:
            break
        ch1 = [set(ch[0]), set(ch[1])]
        before += [ch1]
        best = -1
        bestpair = (-1, -1)
        left.sort(key=lambda a:cost_left[a] - cost_right[a])
        right.sort(key=lambda a:-cost_left[a] + cost_right[a])
        quit = False
        for i in range(1, len(left)):
            if quit:
                break
            for j in range(1, len(right)):
                a = left[i]
                b = right[j]
                now = cur
                now += cost_left[a] - cost_right[a]
                now += -cost_left[b] + cost_right[b]
                oldnow = now
                if best != -1 and now >= min(opt, best): #дальше все варианты будут заведомо хуже, 
                    #в силу сортировки лучшие варианты при меньших i и j
                    quit = True
                    break
                if (a, b) in edges:
                    now += 2
                ch1 = [set(ch[0]), set(ch[1])]
                move(ch1, a, b)
                if ch1 in before: #уже были там в этой ветке
                    continue
                if (best == -1 or now < best):
                    best = now
                    bestpair = (i, j)
        if best == -1:
            break
        i, j = bestpair
        a, b = left[i], right[j]
        left[i], right[j] = right[j], left[i]
        cur = best
        for bb in G[a]:
            if b != bb:
                cost_left[bb] -= 1
                cost_right[bb] += 1
        for aa in G[b]:
            if a != aa:
                cost_left[aa] += 1
                cost_right[aa] -= 1
        if (a, b) in edges:
            cost_left[a] += 1
            cost_right[a] -= 1
            cost_left[b] -= 1
            cost_right[b] += 1
        #print(a, b)
        if best < opt:
            opt = best
            bestleft = left[::-1]
            before = []
            ch = [set(), set()]
            r = 0
        else:
            r += 1
            move(ch, a, b)
            #print("CH", ch)
    return set(bestleft)


In [180]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [181]:
run_all()

Solving instance add20.graph… done in 0.67 seconds with quality 12142


In [102]:
run_all()

Solving instance add20.graph…1927
1920
1914
1907
1902
1897
1892
1887
1882
1877
1872
1867
1862
1857
1852
1847
1842
1837
1832
1827
1822
1817
1812
1807
1802
1797


KeyboardInterrupt: 

## Выводы
(Здесь опишите свои наблюдения и подобранные параметры для каждогр из четырёх входных графов.)

In [ ]:
Работает очень долго, несмотря на оптимизации. Ничего подобрать не удалось. 

In [45]:
run_all()

Solving instance add20.graph… done in 4.5e+01 seconds with quality 12010


In [111]:
run_all()

Solving instance add20.graph…1927
1920
1914
1907
1901
1894
1887
1881
1875
1869
1864
1859
1854
1849
1844
1839
1834
1829
1824
1819
1814
1809
1803
1797
1791
1783
1777
1771
1765
1759
1753
1747
1742
1738
1734
1730
1726
1720
1714
1708
1700
1692
1683
1676
1669
1663
1657
1649
1641
1632
1625
1618
1609
1602
1595
1588
1582
1576
1569
1561
1553
1547
1541
1535
1529
1524
1519
1514
1510
1506
1502
1498
1494
1490
1486
1482
1478
1474
1470
1466
1460
1456
1452
1448
1444
1440
1436
1432
1428
1424
1418
1411
1405
1397
1391
1385
1380
1373
1367
1363
1357
1352
1345
1340
1334
1328
1324
1318
1313
1307
1301
1296
1291
1285
1280
1274
1269
1264
1257
1251
1243
1237
1231
1225
1219
1213
1206
1201
1195
1189
1184
1179
1172
1166
1159
1153
1147
1142
1137
1132
1127
1122
1117
1112
1106
1101
1096
1091
1086
1081
1076
1072
1066
1060
1056
1050
1046
1042
1037
1032
1028
1024
1020
1016
1012
1008
1003
999
995
990
986
981
977
973
969
964
959
954
950
946
942
938
934
929
925
920
915
910
905
900
896
891
888
883
880
877
874
871
868
865
862


In [129]:
print(variable_depth_local_search(({1, 2, 3, 4}, {(1, 2), (2, 1), (1, 3),(3, 1), (1, 4), (4, 1)})))

2
[1, 2]
[0, 1, 1, 1, 1]
[0, 2, 0, 0, 0]
2 4
2
[1, 4]
[0, -1, 1, 1, 1]
[0, 4, 0, 0, 0]
4 2
2
[1, 2]
[0, -3, 1, 1, 1]
[0, 6, 0, 0, 0]
2 4
2
[1, 4]
[0, -5, 1, 1, 1]
[0, 8, 0, 0, 0]
4 2
2
[1, 2]
[0, -7, 1, 1, 1]
[0, 10, 0, 0, 0]
2 4
2
[1, 4]
[0, -9, 1, 1, 1]
[0, 12, 0, 0, 0]
{1, 2}
